# RL Meta RAG Retrieval

RL Meta RAG introduces a higher level of abstraction by employing Reinforcement Learning (specifically a QLearningAgent in neural mode) to dynamically select the most suitable underlying RAG technique (e.g., Llama Index, HyDE, Adaptive RAG) for a given user query. The goal is to learn an optimal policy over time that maximizes retrieval effectiveness based on query characteristics and feedback.

## System Introduction

The RL Meta RAG system acts as an intelligent orchestrator for various RAG retrieval methods. Instead of relying on a single technique, it uses a Reinforcement Learning agent, trained via Q-learning with a Deep Q-Network (DQN), to decide which RAG approach is best suited for the user's query in real-time. This adaptive selection aims to optimize the quality and relevance of the retrieved information, learning from experience and potentially human feedback.

## Underlying Concept

Different RAG techniques excel at different types of queries. Simple queries might be handled well by standard vector search (Llama Index), while more complex or nuanced queries might benefit from hypothetical document generation (HyDE) or adaptive strategies that classify the query first (Adaptive RAG). RL Meta RAG treats the selection of the RAG technique as a sequential decision-making problem. The RL agent learns a policy (a mapping from query features to the best RAG technique) by maximizing expected future rewards, which ideally correlate with retrieval quality.

## Key Features
- **Dynamic RAG Selection:** Utilizes a Deep Q-Network (DQN) agent (`QLearningAgent` in neural mode) to choose the best RAG method (Llama Index, HyDE, Adaptive RAG) based on learned Q-values for the current query's features.
- **LLM-Powered Feature Extraction:** Leverages an LLM (`call_model` with `EXTRACT_QUERY_FEATS_PROMPT`) to analyze the input query and extract numerical features (e.g., query type, complexity, keyword count) plus query length, forming the state representation for the RL agent.
- **Adaptive Strategy (RL vs. LLM Suggestion):** Monitors the RL agent's recent performance by tracking the average TD error over the last `n_recent` episodes. If the error is low (below `error_threshold`), it trusts the agent's learned policy (using epsilon-greedy selection). If the error is high or there's insufficient data, it falls back to an LLM-based suggestion (`llm_suggest_action` using `SUGGEST_ACTION_PROMPT`) for the RAG technique.
- **Human-in-the-Loop Feedback (Optional):** If `request_human_evaluation` is enabled, it stores the state, chosen action (RAG technique), and query in Redis (`rl_update:{session_id}`). It then emits a `request_evaluation` event via Socket.IO, allowing a human user to provide a reward signal. This feedback can be used later to update the Q-learning agent's model, improving its policy based on real user evaluation.
- **Real-time Reasoning Updates:** Uses Socket.IO (`socketio.emit('reasoning_update', ...)` ) to provide frontend visibility into the feature extraction process, the decision logic (RL vs. LLM), Q-values, epsilon exploration choices, chosen RAG technique, and feedback storage/request steps.

## How It Works (Workflow)

### Retrieval Steps:
1. **Start Retrieval:** Initiates the process upon receiving a user query.
2. **Extract Query Features:** Uses an LLM to analyze the query and generate a numerical state vector (`_extract_query_features`).
3. **Choose Action (RAG Technique):** 
   - Checks the average TD error of the Q-agent over recent interactions (`choose_action`).
   - If error is low & sufficient data: Uses the Q-agent's policy (epsilon-greedy selection based on Q-values) to choose an action (RAG technique ID).
   - If error is high or insufficient data: Uses an LLM suggestion (`llm_suggest_action`) to choose the action.
4. **Execute Selected RAG:** Calls the retrieval function corresponding to the chosen action ID (e.g., `_llama_index`, `_hyde_rag`, `_adaptive_rag`).
5. **Generate Final Answer:** Uses the retrieved context and the original query with another LLM call (`RAG_ANSWER_PROMPT`) to synthesize a final response.
6. **Handle Feedback (Optional):**
   - If `request_human_evaluation` is true, stores the state, action, and query in Redis.
   - Emits a `request_evaluation` event via Socket.IO, sending the generated answer and session ID to the frontend for potential human rating.
7. **Return Final Answer:** Outputs the LLM-generated answer to the user.

*(Note: The Q-learning agent's update step, using the reward obtained from human feedback, happens separately, likely triggered by a response to the `request_evaluation` event.)*

## System Advantages

- **Adaptive Retrieval:** Automatically selects the potentially best RAG strategy for diverse queries.
- **Self-Improving:** Can learn and improve its selection policy over time, especially when coupled with human feedback.
- **Optimized Performance:** Aims to provide better retrieval results than using a single fixed RAG technique.
- **Transparency:** Provides real-time reasoning updates via Socket.IO for better understanding of the selection process.
- **Leverages Multiple Strengths:** Combines the pattern recognition of RL with the zero-shot reasoning capabilities of LLMs.

## Use Cases

- Automatically optimizing RAG performance across diverse and unpredictable query types in a chat or search system.
- Building systems that continuously learn and adapt their information retrieval strategy based on implicit or explicit user feedback.
- Creating an intelligent orchestration layer that selects the best tool from a suite of different RAG implementations or other information retrieval tools.
- Researching adaptive AI systems that combine RL and LLMs for complex decision-making.

## Example Prompt

- "Using the `retrieve_rl_meta_rag` tool, find the latest advancements in quantum computing." *(The tool automatically selects the underlying RAG technique based on its learned policy or LLM suggestion.)*

## Implementation Insights

- **rl_meta_rag_retrieve.py:** Contains the core `RlMetaRag` class orchestrating the process.
- **reinforcement_learning/qlearn_agent.py:** Provides the `QLearningAgent` implementation (specifically the neural mode using DQN).
- **tools/rag/rl_meta_rag/prompts.py:** Defines the LLM prompts used for feature extraction (`EXTRACT_QUERY_FEATS_PROMPT`), LLM-based action suggestion (`SUGGEST_ACTION_PROMPT`), and final answer generation (`RAG_ANSWER_PROMPT`).
- **params.py:** Stores configuration parameters like model names, file paths, RL hyperparameters, and feature flags.
- **Individual RAG Modules:** Depends on the retrieval functions within `tools/rag/llama_index/retrieve.py`, `tools/rag/hyde_rag/retrieve.py`, and `tools/rag/adaptive_rag/retrieve.py`.

## Parameters (from `params.py`)

- **RL_META_RAG_MODEL:** LLM used for feature extraction, action suggestion, and final answer generation.
- **RAG_Q_DATA_FILE:** File path to save/load the Q-learning agent's training data (weights, memory).
- **RL_N_RECENT:** Number of recent TD errors to consider for the adaptive strategy.
- **RL_ERROR_THRESHOLD:** Threshold for the average TD error to switch between RL policy and LLM suggestion.
- **RL_ALPHA:** Learning rate for the Q-learning agent.
- **RL_GAMMA:** Discount factor for the Q-learning agent.
- **RL_EPSILON:** Exploration rate (epsilon) for the epsilon-greedy policy in the Q-learning agent.
- **RL_REQUEST_HUMAN_EVALUATION:** Boolean flag to enable/disable the human feedback loop via Redis and Socket.IO.

## Conclusion

RL Meta RAG represents a sophisticated approach to information retrieval by dynamically selecting the most appropriate RAG technique using Reinforcement Learning. It intelligently combines an RL agent's learned policy with LLM suggestions, adapting its strategy based on performance. The optional human-in-the-loop feedback mechanism allows for continuous improvement, making it a powerful tool for building adaptive and optimized retrieval systems. This meta-approach allows the system to leverage the strengths of different RAG methods and adapt its strategy intelligently, moving towards a more robust and self-improving information retrieval system.